# Segmenting and Clustering Neighborhoods in Toronto

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Question 1</a>
    
1.1. <a href="#item1">Conclusion</a>
2. <a href="#item2">Question 2</a>
    
2.1. <a href="#item1">Conclusion</a>
    
3. <a href="#item3">Question 3</a>
    
3.1. <a href="#item1">Conclusion</a>
</font>
</div>

## 1. Question 1

*Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:*

<img src = "https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/7JXaz3NNEeiMwApe4i-fLg_40e690ae0e927abda2d4bde7d94ed133_Screen-Shot-2018-06-18-at-7.17.57-PM.png?expiry=1588636800000&hmac=ssKIQrsG6VHIIby2_yiH4jQ1yUt124BPn_UWPv6ncGk" width="500" height="500" />

The code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

(optional) If needed, install the *pip* package in the current Jupyter kernel.

In [1]:
# import sys
# !{sys.executable} -m pip install BeautifulSoup4

Scrape the webpage from Wikipedia that contains the complete list of postal codes for the city of Toronto. We use the *request* library to scrape the page.

In [2]:
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

We will use  the *BeautifulSoup* library to handle the content of the page. We will use the *lxml* parser since its the recommended one and also reportedly very fast.

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')

Let's look for a table in the code. *BeautifulSoup* allows us to search for an HTML element by it's type. So lets look for a *table*.

In [4]:
soup.table.name

'table'

There is one table element named 'table' in the text. Let's get the first table row (the first *tr* element) to see if this has table headers.

In [5]:
#another way to get the same result: soup.table.tr.findAll()
print(soup.table.tr.text)


Postal code

Borough

Neighborhood



So now lets scrape the postal codes to a Data Frame. We will iterate in the table and look for all the *tr* elements, the rows and scrape the data from it. Since the first row contains the row header we will need to remove it from the data frame after all the data is loaded.

In [6]:
import pandas as pd

The list of headers for our data frame.

In [67]:
headers=["Postalcode","Borough","Neighborhood"]

The data frame is named *df_toronto*.

In [68]:
df_toronto = pd.DataFrame(columns= headers)

In [69]:
for row in soup.table.find_all('tr'):
    row_data=[]
    for data in row.find_all('th'):
        row_data.append(data.text.strip())
    for data in row.find_all('td'):
        row_data.append(data.text.strip())
    df_toronto.loc[len(df_toronto)] = row_data

Now we have our data frame, let's check it.

In [70]:
df_toronto.head()

,Postalcode,Borough,Neighborhood
0,Postal code,Borough,Neighborhood
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


We need to remove the first row from the Dataset.

In [71]:
# delete the first row from the dataFrame
df_toronto.drop(0, inplace=True)

In [72]:
df_toronto.head()

,Postalcode,Borough,Neighborhood
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront


Now we will remove the postal codes not assigned to a borough.

In [73]:
# Get names of indexes for which the column Borough has a value "Not assigned"
not_assigned = df_toronto[df_toronto['Borough'] =='Not assigned'].index

# Delete the row indexes from the data frame
df_toronto.drop(not_assigned, inplace=True)

In [74]:
df_toronto.head()

,Postalcode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront
6,M6A,North York,Lawrence Manor / Lawrence Heights
7,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


Let's check how many rows the data frame contains now.

In [75]:
df_toronto.shape

(103, 3)

How many distinct postal codes are in the data frame?

In [76]:
df_toronto.nunique()

Postalcode      103
Borough          10
Neighborhood     98
dtype: int64

So there is no need to merge different rows since this version of the page has no longer repeated postal codes. But the neighbourhood names are not separeted by commas but by slashes /. So let's fix this.

In [78]:
df_toronto.Neighborhood = df_toronto.Neighborhood.replace(" /", ',', regex=True)
df_toronto.head()

,Postalcode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### 1.1. Conclusion

We can see the full data frame below.

In [79]:
df_toronto

,Postalcode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
9,M9A,Etobicoke,Islington Avenue
10,M1B,Scarborough,"Malvern, Rouge"
12,M3B,North York,Don Mills
13,M4B,East York,"Parkview Hill, Woodbine Gardens"
14,M5B,Downtown Toronto,"Garden District, Ryerson"


## 2. Question 2

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Use the Geocoder package or the csv file to create the following dataframe:
<img src="https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/HZ3jNHNOEeiMwApe4i-fLg_f44f0f10ccfaf42fcbdba9813364e173_Screen-Shot-2018-06-18-at-7.18.16-PM.png?expiry=1588636800000&hmac=epN7y9Ean0VJJ-40CNmcZ9ztJhgYTUYI5v09sgxi9WY" width="500" height="500" />

There was no data retrieved from Google using the *geocoder* library. *Geocoder* offers other geodata sources, for example *arcgis* for the same purpose.

I will first retrieve the coordinates data from the csv file available for this question.

In [80]:
#add Geo-spatial data
df_coord= pd.read_csv("http://cocl.us/Geospatial_data")
#dfll.set_index("Postcode")

We can check that the file was loaded in the data frame.

In [81]:
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We need to rename the first column to *Postalcode*  so we can merge this data frame with the *boroughs* dataframe we previously created, *df_toronto*.

In [82]:
df_coord.rename(columns={'Postal Code':'Postalcode'},inplace=True)

This dataframe has the same number of rows.

In [83]:
df_coord.shape

(103, 3)

Lets merge the two datagrames, using the *Postalcode* column as the column name to join on.

In [84]:
df_all_boroughs = pd.merge(df_toronto, df_coord, on = 'Postalcode')

The result of the merge is the dataframe *df_all_boroughs*.

In [85]:
df_all_boroughs.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Retrieving the geo coordinates from the ArcGis database using the *geocoder* library

Since using geocoder to retrieve geo coordinates from Google was not working, we used another well known geo data provider, *ArcGis*. 
We don't need an API key to use it.

First we need to install *geocoder* and import it.

In [86]:
import sys
!{sys.executable} -m pip install geocoder

In [87]:
import geocoder

Using *geocoder* to retrieve geo data from Google is not working:

In [88]:
import geocoder
g = geocoder.google('Mountain View, CA')
g.latlng
print(g.latlng)

None


So we will create a function to retrieve geo data from ArcGis using *geocoder*.

In [89]:
def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords[0],lat_lng_coords[1]

If we feed the function with an address we will get the geo data for it.

In [90]:
get_latlng('M3A')

(43.75293455500008, -79.33564142299997)

We will create a new column on the data frame and fill it with the geo data for the postal code in each line.

In [91]:
df_toronto['coord'] = df_toronto.Postalcode.apply(lambda x: get_latlng(x))

The data frame now has all the geo data we need but now we need to split it in two columns, *Latitude* and *Longitude* and drop the *coord* column after it.

In [92]:
df_toronto.head()

,Postalcode,Borough,Neighborhood,coord
3,M3A,North York,Parkwoods,"(43.75293455500008, -79.33564142299997)"
4,M4A,North York,Victoria Village,"(43.72810248500008, -79.31188987099995)"
5,M5A,Downtown Toronto,"Regent Park, Harbourfront","(43.65096410900003, -79.35304116399999)"
6,M6A,North York,"Lawrence Manor, Lawrence Heights","(43.723265465000054, -79.45121077799996)"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government","(43.66179000000005, -79.38938999999993)"


In [93]:
df_toronto['Latitude'] = df_toronto.coord.apply(lambda x: x[0])
df_toronto['Longitude'] = df_toronto.coord.apply(lambda x: x[1])

In [94]:
df_toronto.drop("coord", axis=1, inplace=True)

Finally, we have the data frame with the desired data, labeled as we wanted.

We can see that both data frames have the same data, with slight differences only on latitude and longitude values can be explained by the different option how to define the coordinates of a neighbourhood. 

In [95]:
df_toronto.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
3,M3A,North York,Parkwoods,43.752935,-79.335641
4,M4A,North York,Victoria Village,43.728102,-79.311890
5,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
6,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390


In [96]:
df_toronto.nunique()

Postalcode      103
Borough          10
Neighborhood     98
Latitude        102
Longitude       102
dtype: int64

In [97]:
df_toronto[df_toronto.Neighborhood.duplicated()]

,Postalcode,Borough,Neighborhood,Latitude,Longitude
21,M3C,North York,Don Mills,43.722143,-79.352023
75,M3L,North York,Downsview,43.729992,-79.512027
84,M3M,North York,Downsview,43.733220,-79.497700
93,M3N,North York,Downsview,43.755819,-79.519973
110,M2R,North York,Willowdale,43.779912,-79.445229


In [103]:
df_all_boroughs.nunique()

Postalcode      103
Borough          10
Neighborhood     98
Latitude        103
Longitude        75
dtype: int64

In [98]:
df_all_boroughs[df_all_boroughs.Neighborhood.duplicated()]

,Postalcode,Borough,Neighborhood,Latitude,Longitude
13,M3C,North York,Don Mills,43.725900,-79.340923
46,M3L,North York,Downsview,43.739015,-79.506944
53,M3M,North York,Downsview,43.728496,-79.495697
60,M3N,North York,Downsview,43.761631,-79.520999
72,M2R,North York,Willowdale,43.782736,-79.442259


### 2.1. Conclusion

We retrieved the geo data from two sources. The first source was the CSV file provided by IBM and the second source was the geo data from ArcGis retrieved using *geocoder*.

In [99]:
# from a CSV file
df_all_boroughs

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [100]:
# from the ArcGis geo data using the geocoder library
df_toronto

,Postalcode,Borough,Neighborhood,Latitude,Longitude
3,M3A,North York,Parkwoods,43.752935,-79.335641
4,M4A,North York,Victoria Village,43.728102,-79.311890
5,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
6,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390
9,M9A,Etobicoke,Islington Avenue,43.667481,-79.528953
10,M1B,Scarborough,"Malvern, Rouge",43.808626,-79.189913
12,M3B,North York,Don Mills,43.748900,-79.357220
13,M4B,East York,"Parkview Hill, Woodbine Gardens",43.707193,-79.311529
14,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529


## 3. Question 3

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

to add enough Markdown cells to explain what you decided to do and to report any observations you make.
to generate maps to visualize your neighborhoods and how they cluster together.
Once you are happy with your analysis, submit a link to the new Notebook on your Github repository.

In [62]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                       

## 1. Explore Toronto and select the neighborhoods to cluster

Let first display a map of all Toronto neighborhoods.

In [101]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the city of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of the city of Toronto are 43.6534817, -79.3839347.


In [105]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

We will focus on the downton of Toronto. We will cluster only the neighbourhoods included in the Toronto Downtown borough.

In [109]:
downtown_data = df_toronto[df_toronto['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529
3,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554
4,M5E,Downtown Toronto,Berczy Park,43.645196,-79.373855


Display now the map zooming in Downtown Toronto.

In [110]:
address = 'Downtown Toronto, Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [116]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

Foursquare Api credentials

In [118]:
CLIENT_ID = 'XEJXIC4ABL0SSRAWKMYJ5UKIO2DRR0HIU14KOCR5YIHZR2MP' # your Foursquare ID
CLIENT_SECRET = 'P0L3PE2EPASSSHZS513AFA15UNOARCAMHXVRS1ZOUYKOMG3R' # your Foursquare Secret

# needed now
ACCESS_TOKEN = 'CIIQUWECMQFQIEDQW3IKZUNBIJTLCXFW4DFQAVX13CZBRQBD'
VERSION = '20180605' # Foursquare API version

#### Exploring the geo data

Get the first neighborhood in the dataset and explore the information we can get from Foursquare related to it.

In [120]:
downtown_data.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

Get the neighborhood's latitude and longitude values.

In [121]:
neighborhood_latitude = downtown_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = downtown_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = downtown_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.65096410900003, -79.35304116399999.


In [122]:
#### Exploring the venues around it. Lets use a 500 metres around the latitude and longitude we have.

In [124]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eb0a3cc216785001b209b1a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Downtown Toronto',
  'headerFullLocation': 'Downtown Toronto, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 24,
  'suggestedBounds': {'ne': {'lat': 43.65546411350003,
    'lng': -79.34683350482591},
   'sw': {'lat': 43.646464104500026, 'lng': -79.35924882317407}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '57e0745a498ea809dbf75f68',
       'name': 'Souk Tabule',
       'location': {'address': '494 Front St E',
        'crossStreet': 'at Bayview',
        'lat': 43.65375556880743,
        'lng': -79.35439006096168,
        'labeledLatLngs

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [125]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [127]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Souk Tabule,Mediterranean Restaurant,43.653756,-79.354390
1,Young Centre for the Performing Arts,Performing Arts Venue,43.650825,-79.357593
2,SOMA chocolatemaker,Chocolate Shop,43.650622,-79.358127
3,BATLgrounds,Athletics & Sports,43.647088,-79.351306
4,Cluny Bistro & Boulangerie,French Restaurant,43.650565,-79.357843


In [128]:
print('We have a total of {} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

We have a total of 24 venues were returned by Foursquare.


## 3.2. Explore Neighborhoods in Downtown Toronto

So now we will do the same data collection for all the neighborhoods in our selection.

First we define a function to retrive the data for each neighborhood.

In [131]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Using this fuction we create a new dataframe with all the venues in a 500 meters radius of each neighborhood. The dataframe is called *downtown_venues*.

In [134]:
downtown_venues = getNearbyVenues(names=downtown_data['Neighborhood'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


#### Lets check the size of the resulting dataframe

In [136]:
print(downtown_venues.shape)
downtown_venues.head()

(1157, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.650964,-79.353041,Souk Tabule,43.653756,-79.354390,Mediterranean Restaurant
1,"Regent Park, Harbourfront",43.650964,-79.353041,Young Centre for the Performing Arts,43.650825,-79.357593,Performing Arts Venue
2,"Regent Park, Harbourfront",43.650964,-79.353041,SOMA chocolatemaker,43.650622,-79.358127,Chocolate Shop
3,"Regent Park, Harbourfront",43.650964,-79.353041,BATLgrounds,43.647088,-79.351306,Athletics & Sports
4,"Regent Park, Harbourfront",43.650964,-79.353041,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant


Grouping the dataframe by neighborhood we can see how many venues are in each neighborhood.

In [137]:
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,66,66,66,66,66,66
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",64,64,64,64,64,64
Central Bay Street,54,54,54,54,54,54
Christie,12,12,12,12,12,12
Church and Wellesley,86,86,86,86,86,86
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",7,7,7,7,7,7


#### How many unique categories can be curated from all the returned venues

In [139]:
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 185 uniques categories.


## 3. Analyze Each Neighborhood

#### We now create a dataframe will all the venues and the total of each venue by neighborhood.

##### After we create the new dataframe we need to rearrange it so the neighborhood name appears as the first column, for that we dropt the column and then we add it from the original data frame.

In [182]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# drop the Neighborhood column (that doesn't have the names at the moment)
downtown_onehot = downtown_onehot.drop(['Neighborhood'], axis = 1)

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]]  + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Neighborhood,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

##### This data frame is huge, we can see that there are now the same number of rows we fetched before but the number of colunmns are now the same of the total of distinct venues we retrieved.

In [179]:
downtown_onehot.shape

(1157, 185)

In [ ]:
#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [183]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.015152,0.000000,0.000000,0.000000,0.000000,0.015152,0.000000,0.015152,0.030303,0.000000,0.000000,0.015152,0.030303,0.000000,0.00000,0.000000,0.015152,0.000000,0.00,0.030303,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.030303,0.000000,0.000000,0.00,0.015152,0.045455,0.090909,0.000000,0.000,0.000000,0.00,0.00,0.015152,0.000000,0.015152,0.000000,0.015152,0.00,0.015152,0.00,0.000000,0.015152,0.015152,0.000000,0.015152,0.000,0.000000,0.000000,0.000000,0.015152,0.00,0.000000,0.000000,0.000000,0.015152,0.015152,0.000000,0.015152,0.000000,0.000000,0.000000,0.015152,0.015152,0.015152,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.015152,0.015152,0.015152,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.030303,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.015152,0.015152,0.000000,0.015152,0.00,0.000000,0.000000,0.015152,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.015152,0.000000,0.000000,0.00,0.015152,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.00,0.000000,0.015152,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015152,0.00,0.015152,0.000000,0.00000,0.030303,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.015152,0.000000,0.000000,0.00,0.00,0.

#### Grouping by *Neighborhood* provied a new data frame with the 19 neighborhoods and the number of rows equal to the number of unique venues we found earlier. The data in the set is now showing us the relative size of each type of venue in each neighborhood. Check this in the table above and the dimension in the line below.

In [185]:
downtown_grouped.shape

(19, 185)

#### Let's print each neighborhood along with the top 5 most common venues

In [213]:
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2        Lounge  0.03
3        Bakery  0.03
4    Restaurant  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
               venue  freq
0        Coffee Shop  0.06
1               Café  0.06
2               Park  0.05
3  French Restaurant  0.05
4         Restaurant  0.05


----Central Bay Street----
                       venue  freq
0                Coffee Shop  0.15
1                       Café  0.06
2  Middle Eastern Restaurant  0.06
3         Italian Restaurant  0.04
4            Bubble Tea Shop  0.04


----Christie----
           venue  freq
0           Café  0.25
1  Grocery Store  0.25
2     Playground  0.08
3     Baby Store  0.08
4    Coffee Shop  0.08


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.13
1  Japanese Restaurant  0.07
2     Sushi Restaurant  0.05
3           Restaurant  0.

#### Once again, we create *pandas* dataframe with this data

First, let's write a function to sort the venues in descending order.

In [214]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now we create the new dataframe and display the top 10 venues for each neighborhood.

In [221]:
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Lounge,Bakery,Café,Seafood Restaurant,Breakfast Spot,Cheese Shop,Beer Bar,Restaurant,Hotel,Fast Food Restaurant,Museum,Pub,Gourmet Shop
1,"CN Tower, King and Spadina, Railway Lands, Har...",Café,Coffee Shop,Restaurant,French Restaurant,Park,Bar,Speakeasy,Japanese Restaurant,Italian Restaurant,Lounge,Gym / Fitness Center,Falafel Restaurant,Dance Studio,Grocery Store,Ramen Restaurant
2,Central Bay Street,Coffee Shop,Middle Eastern Restaurant,Café,Breakfast Spot,Restaurant,Clothing Store,Plaza,Sandwich Place,Bubble Tea Shop,Italian Restaurant,Bookstore,Shopping Mall,Shoe Store,Fast Food Restaurant,Japanese Restaurant
3,Christie,Café,Grocery Store,Park,Coffee Shop,Candy Store,Playground,Athletics & Sports,Baby Store,Farmers Market,Farm,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Pub,Grocery Store,Smoke Shop,Hotel,Dance Studio,Gay Bar,Gastropub,Café,Mediterranean Restaurant,Men's Store,Martial Arts Dojo
5,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Hotel,Café,Italian Restaurant,American Restaurant,Japanese Restaurant,Gym,Deli / Bodega,Seafood Restaurant,Cocktail Bar,Tea Room,Breakfast Spot,Beer Bar,Gastropub
6,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,American Restaurant,Gym,Japanese Restaurant,Restaurant,Deli / Bodega,Asian Restaurant,Steakhouse,Seafood Restaurant,Gastropub,Beer Bar,Thai Restaurant,Concert Hall
7,"Garden District, Ryerson",Coffee Shop,Clothing Store,Sandwich Place,Middle Eastern Restaurant,Hotel,Restaurant,Café,Bar,Italian Restaurant,Cosmetics Shop,Tea Room,Tanning Salon,Diner,Ramen Restaurant,Movie Theater
8,"Harbourfront East, Union Station, Toronto Islands",Harbor / Marina,Theme Park,Fast Food Restaurant,Park,Farm,Distribution Center,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Dog Run,Discount Store,Fish Market
9,"Kensington Market, Chinatown, Grange Park",Café,Mexican Restaurant,Coffee Shop,Bakery,Vietnamese Restaurant,Gaming Cafe,Vegetarian / Vegan Restaurant,Cheese Shop,Record Shop,Cocktail Bar,Pizza Place,Park,Organic Grocery,Comfort Food Restaurant,Music Venue


## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [225]:
# set number of clusters
kclusters = 7

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 3, 1, 3, 3, 3, 3, 2, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [226]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
downtown_merged = downtown_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head() # check the last columns!

ValueError: cannot insert Cluster Labels, already exists

Finally, let's visualize the resulting clusters

In [224]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=14)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters